# Carregando Google Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=False)

Mounted at /content/drive


# Checando GPU

In [2]:
!nvidia-smi

Wed Sep 21 19:44:54 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

#Carregando Pacotes

In [3]:
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet import preprocess_input
from pathlib import Path
import shutil

# ResNet50

https://keras.io/api/applications/ 

https://keras.io/api/applications/resnet/#resnet50-function

In [4]:
resnet50 = ResNet50()
resnet50.summary()

102981632/102967424 [==============================] - 1s 0us/step
Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                        

In [5]:
resnet50_top_false =  ResNet50(include_top=False, input_shape=(224,224,3))
resnet50_top_false.summary()

94781440/94765736 [==============================] - 1s 0us/step
Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_2[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                          

# Paramêtros

In [6]:
# Tamanho imagem
img_rows, img_cols = 224, 224

input_shape = (img_rows, img_cols, 3)

#Batch size

batch_size = 32

#Path base de dados Drive
nome_base = 'oregon_wildlife'
nome_zip = nome_base + '.zip'
dir_path = Path('/content/drive/MyDrive/CI1394/')
zip_path = dir_path/nome_zip

#Arquivos npy
nome_modelo = 'resnet50'
!mkdir {nome_modelo}
npy_X_train = nome_modelo + '/X_train.npy' 
npy_y_train = nome_modelo + '/y_train.npy'  
npy_X_test = nome_modelo + '/X_test.npy'
npy_y_test = nome_modelo + '/y_test.npy'

#Arquivos zip

zip_name = nome_modelo + '.zip'
path_zip_caracteristicas = dir_path/zip_name


#Carregar Base de Dados

In [7]:
!cp {zip_path} .
!unzip -q {nome_zip} # Utilizar as imagens diretamente do Drive é extremamente lento
!rm {nome_zip}

In [8]:
path_train = '/content/' + nome_base + '/train'
path_test = '/content/' + nome_base + '/test'

In [9]:
train_dataset = image_dataset_from_directory(
        path_train,
        image_size=(img_rows, img_cols),
        color_mode="rgb",
        batch_size=batch_size,
        shuffle=False) # Manter Shuffle = False, pois não haverá treinamento
test_dataset = image_dataset_from_directory(
        path_test,
        image_size=(img_rows, img_cols),
        color_mode="rgb",
        batch_size=batch_size,
        shuffle=False)

Found 3136 files belonging to 14 classes.
Found 784 files belonging to 14 classes.


In [10]:
AUTOTUNE = tf.data.AUTOTUNE

train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE) #Otimização
test_dataset = test_dataset.prefetch(buffer_size=AUTOTUNE)

# Criando o extrator

https://keras.io/guides/transfer_learning/

In [11]:
cnn = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
inputs = keras.Input(shape=input_shape)
x = preprocess_input(inputs)
x = cnn(x)
output = GlobalAveragePooling2D()(x)
model = Model(inputs, output)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 tf.__operators__.getitem (S  (None, 224, 224, 3)      0         
 licingOpLambda)                                                 
                                                                 
 tf.nn.bias_add (TFOpLambda)  (None, 224, 224, 3)      0         
                                                                 
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 global_average_pooling2d (G  (None, 2048)             0         
 lobalAveragePooling2D)                                          
                                                                 
Total params: 23,587,712
Trainable params: 23,534,592
Non-tra

# Extraindo Características

In [12]:
X_train = model.predict(train_dataset)
X_test = model.predict(test_dataset)

In [14]:
print(X_train.shape)
print(X_test.shape)

(3136, 2048)
(784, 2048)


In [15]:
y_train = np.concatenate([y for x, y in train_dataset], axis=0)
y_test = np.concatenate([y for x, y in test_dataset], axis=0)

In [16]:
print(y_train.shape)
print(y_test.shape)

(3136,)
(784,)


In [17]:
np.save(npy_X_train, X_train)
np.save(npy_y_train, y_train)
np.save(npy_X_test, X_test)
np.save(npy_y_test, y_test)

# Copiando para o Drive

In [18]:
!zip -r {zip_name} {nome_modelo}

  adding: resnet50/ (stored 0%)
  adding: resnet50/X_train.npy (deflated 15%)
  adding: resnet50/X_test.npy (deflated 15%)
  adding: resnet50/y_train.npy (deflated 99%)
  adding: resnet50/y_test.npy (deflated 96%)


In [19]:
shutil.copyfile(zip_name, path_zip_caracteristicas)

PosixPath('/content/drive/MyDrive/CI1394/resnet50.zip')